In [1]:
!pip install torchviz

  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nv

In [2]:
# Import necessary libraries
from google.colab import drive
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2Model, GPT2Config, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc, confusion_matrix
from torchviz import make_dot

# Mount Google Drive
drive.mount('/content/drive')



Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2Model, GPT2Config, get_linear_schedule_with_warmup
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, confusion_matrix
import matplotlib.pyplot as plt
from torch.optim import AdamW





In [4]:
# Step 1: Load and preprocess the dataset
data = pd.read_csv('/content/drive/MyDrive/Training_Essay_Data00.csv', encoding='latin1')  # Load your dataset
data['text'] = data['text'].apply(lambda x: x.lower())  # Convert text to lowercase
data['text'] = data['text'].str.replace('[^a-zA-Z\s]', '')  # Remove special characters and numbers
data['text'] = data['text'].str.replace(r'\s+', ' ')  # Remove extra spaces


In [5]:
# Step 2: Split the dataset into train and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data['text'], data['generated'], test_size=0.2, random_state=42)



In [6]:
# Step 3: Tokenization
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')  # Initialize GPT-2 tokenizer
tokenizer.pad_token = tokenizer.eos_token  # Set padding token to end-of-sequence token

train_encodings = tokenizer(X_train.tolist(), truncation=True, padding=True, max_length=128)  # Tokenize train data
test_encodings = tokenizer(X_test.tolist(), truncation=True, padding=True, max_length=128)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [7]:
# Step 4: Convert data to PyTorch tensors
X_train_input_ids = torch.tensor(train_encodings['input_ids'])
X_train_attention_mask = torch.tensor(train_encodings['attention_mask'])
y_train = torch.tensor(y_train.values)  # Convert y_train to PyTorch tensor

X_test_input_ids = torch.tensor(test_encodings['input_ids'])
X_test_attention_mask = torch.tensor(test_encodings['attention_mask'])
y_test = torch.tensor(y_test.values)  # Convert y_test to PyTorch tensor



In [8]:
# Step 5: Define Dataset class
class CustomDataset(Dataset):
    def __init__(self, input_ids, attention_mask, labels):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_mask[idx],
            'labels': self.labels[idx]
        }



In [9]:
# Step 6: Convert data to PyTorch DataLoader
train_dataset = CustomDataset(X_train_input_ids, X_train_attention_mask, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_dataset = CustomDataset(X_test_input_ids, X_test_attention_mask, y_test)
test_loader = DataLoader(test_dataset, batch_size=32)



In [10]:
# Step 7: Define GPT model architecture
class GPTClassifier(nn.Module):
    def __init__(self, gpt_model):
        super(GPTClassifier, self).__init__()
        self.gpt_model = gpt_model
        self.fc = nn.Linear(gpt_model.config.n_embd, 2)  # Output layer with 2 classes (binary classification)

    def forward(self, input_ids, attention_mask):
        outputs = self.gpt_model(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0, :]  # Extract the pooled output from GPT model
        logits = self.fc(pooled_output)  # Pass pooled output through linear layer
        return logits


In [12]:


# Step 8: Initialize device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Step 8: Initialize GPT model and classifier
config = GPT2Config.from_pretrained('gpt2', pad_token_id=tokenizer.pad_token_id)
gpt_model = GPT2Model.from_pretrained('gpt2', config=config)
model = GPTClassifier(gpt_model).to(device)


In [13]:

# Step 9: Define optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=2e-5)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_loader) * 10)


In [14]:

# Step 10: Training the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
criterion = nn.CrossEntropyLoss()

num_epochs = 10


In [ ]:

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss:.4f}")



Epoch 1/10, Loss: 385.3769
Epoch 2/10, Loss: 318.5089


In [ ]:
# Step 11: Evaluation
model.eval()
y_true = []
y_pred_prob = []



In [ ]:
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].numpy()

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs, dim=1).cpu().numpy()

        y_true.extend(labels)
        y_pred_prob.extend(predictions)



In [ ]:
# Step 12: Calculate evaluation metrics
accuracy = accuracy_score(y_true, y_pred_prob)
precision = precision_score(y_true, y_pred_prob)
recall = recall_score(y_true, y_pred_prob)
f1 = f1_score(y_true, y_pred_prob)
auc = roc_auc_score(y_true, y_pred_prob)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print(f"AUC: {auc}")



In [ ]:

# Step 13: Plot ROC curve
fpr, tpr, thresholds = roc_curve(y_true, y_pred_prob)
roc_auc = auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.show()


In [ ]:
# Step 14: Calculate and plot confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred_prob)
plt.figure(figsize=(8, 6))
plt.imshow(conf_matrix, cmap='Blues', interpolation='nearest')
plt.title('Confusion Matrix')
plt.colorbar()
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.xticks(np.arange(2), ['Non Generated', 'Generated'])
plt.yticks(np.arange(2), ['Non Generated', 'Generated'])
for i in range(2):
    for j in range(2):
        plt.text(j, i, str(conf_matrix[i, j]), horizontalalignment='center', color='white' if conf_matrix[i, j] > conf_matrix.max() / 2 else 'black')
plt.show()


In [ ]:
!pip install onnx onnxruntime onnx-simplifier netron

In [ ]:
import torch
import torch.nn as nn
from transformers import GPT2ForSequenceClassification, GPT2Tokenizer
import onnx
import netron

# Initialize the model and tokenizer
model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2ForSequenceClassification.from_pretrained(model_name)

# Set the model to evaluation mode
model.eval()

# Create a dummy input tensor for the model
dummy_input = torch.randint(0, tokenizer.vocab_size, (1, 128))  # Example input

# Export the model to ONNX format
onnx_path = 'gpt_model.onnx'
torch.onnx.export(
    model,
    (dummy_input, torch.ones(dummy_input.shape, dtype=torch.int64)),  # (input_ids, attention_mask)
    onnx_path,
    verbose=True,
    input_names=['input_ids', 'attention_mask'],
    output_names=['logits'],
    dynamic_axes={'input_ids': {0: 'batch_size', 1: 'sequence_length'},
                  'attention_mask': {0: 'batch_size', 1: 'sequence_length'},
                  'logits': {0: 'batch_size'}}
)


In [ ]:
!netron gpt_model.onnx
